<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Account-Endpoints/blob/main/Get_Account_Profit_and_Loss_Fund_Flow_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 取得帳戶盈虧資金流向 \
# 查詢當前帳戶下永續合約的資金流動情況。 \

In [ ]:
# GET /openApi/swap/v2/user/income \
# 頻率限制：依UID限制為每秒5次 & 依IP組合限制數量 \
# API KEY權限：永續合約交易

In [ ]:
# 若未傳送 startTime 與 endTime，僅返回過去7天的資料。
# 若未傳送 incomeType，則返回所有類型的帳戶盈虧資金流向。
# 資料只保留最近3個月。

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API基礎網址
APIKEY = ""  # 填入你的API Key
SECRETKEY = ""  # 填入你的Secret Key

In [ ]:
def demo():
    payload = {}  # GET請求無內容
    path = '/openApi/swap/v2/user/income'  # 查詢帳戶盈虧資金流向API路徑
    method = "GET"  # HTTP請求方法

    paramsMap = {
        "startTime": "1702713615001",  # 查詢起始時間，毫秒時間戳
        "endTime": "1702731787011",    # 查詢結束時間，毫秒時間戳
        "limit": "1000"                # 返回資料數量限制
    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉成URL查詢字串，並加上timestamp
    return send_request(method, path, paramsStr, payload)  # 發送API請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 利用HMAC-SHA256演算法對payload字串產生簽名
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名供除錯
    return signature

In [ ]:
def send_request(method, path, urlpa, payload):
    # 組合完整請求URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出請求URL方便調試

    headers = {
        'X-BX-APIKEY': APIKEY,  # API Key放HTTP Header
    }

    response = requests.request(method, url, headers=headers, data=payload)  # 發送HTTP請求
    return response.text  # 回傳API回應內容


In [ ]:
def parseParam(paramsMap):

    # 將參數依鍵排序並組合成URL參數字串
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    # 加入毫秒時間戳，防止重放攻擊
    if paramsStr != "":
        return paramsStr + "&timestamp=" + str(int(time.time() * 1000))
    else:
        return "timestamp=" + str(int(time.time() * 1000))


In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行示範並印出結果